In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# Extract CSVs into DataFrames
covid_file = "inpatient_covid.csv"
covid_df = pd.read_csv(covid_file)
covid_df.head()

,state,collection_date,Inpatient Beds Occupied by COVID-19 Patients Estimated,Count LL,Count UL,Percentage of Inpatient Beds Occupied by COVID-19 Patients Estimated,Percentage LL,Percentage UL,Total Inpatient Beds,Total LL,Total UL
0,CW,2020-08-20,"51,374","51,028","51,720",6.92,6.56,7.29,"708,074","705,402","710,746"
1,CW,2020-08-21,"50,250","49,889","50,611",6.77,6.35,7.20,"708,769","705,191","712,347"
2,CW,2020-08-22,"48,325","48,042","48,607",6.53,6.20,6.85,"706,157","703,059","709,255"
3,CW,2020-08-23,"47,924","47,543","48,306",6.49,6.06,6.93,"704,251","701,096","707,406"
4,CW,2020-08-24,"48,416","48,065","48,767",6.51,6.10,6.92,"710,090","707,303","712,877"


In [3]:
drop_col = covid_df.drop(columns={'Count LL', 'Count UL','Percentage LL', 'Percentage UL', 'Total LL', 'Total UL'})
drop_col

,state,collection_date,Inpatient Beds Occupied by COVID-19 Patients Estimated,Percentage of Inpatient Beds Occupied by COVID-19 Patients Estimated,Total Inpatient Beds
0,CW,2020-08-20,"51,374",6.92,"708,074"
1,CW,2020-08-21,"50,250",6.77,"708,769"
2,CW,2020-08-22,"48,325",6.53,"706,157"
3,CW,2020-08-23,"47,924",6.49,"704,251"
4,CW,2020-08-24,"48,416",6.51,"710,090"
...,...,...,...,...,...
1691,WY,2020-09-18,32,2.12,"1,511"
1692,WV,2020-09-19,216,4.03,"5,364"
1693,WY,2020-09-19,34,2.22,"1,511"
1694,WV,2020-09-20,216,4.05,"5,328"


In [10]:
# Transform DataFrame

# Filter DataFrame from specific columns
rename_cols = drop_col.rename(columns={"Inpatient Beds Occupied by COVID-19 Patients Estimated":"INPT Bed Estimate",
                                      "Percentage of Inpatient Beds Occupied by COVID-19 Patients Estimated":"PCT INPT Bed Estimate"})
rename_cols

,state,collection_date,INPT Bed Estimate,PCT INPT Bed Estimate,Total Inpatient Beds
0,CW,2020-08-20,"51,374",6.92,"708,074"
1,CW,2020-08-21,"50,250",6.77,"708,769"
2,CW,2020-08-22,"48,325",6.53,"706,157"
3,CW,2020-08-23,"47,924",6.49,"704,251"
4,CW,2020-08-24,"48,416",6.51,"710,090"
...,...,...,...,...,...
1691,WY,2020-09-18,32,2.12,"1,511"
1692,WV,2020-09-19,216,4.03,"5,364"
1693,WY,2020-09-19,34,2.22,"1,511"
1694,WV,2020-09-20,216,4.05,"5,328"


In [46]:
fifty_states = rename_cols[(rename_cols.state != 'CW' ) & (rename_cols.state != 'PR') & (rename_cols.state != 'DC')]
fifty_states

,state,collection_date,INPT Bed Estimate,PCT INPT Bed Estimate,Total Inpatient Beds
64,AL,2020-08-20,"1,368",9.69,"14,128"
65,AR,2020-08-20,695,9.24,"7,521"
66,CT,2020-08-20,169,2.09,"8,080"
67,ID,2020-08-20,168,4.77,"3,520"
68,IN,2020-08-20,861,5.22,"16,137"
...,...,...,...,...,...
1691,WY,2020-09-18,32,2.12,"1,511"
1692,WV,2020-09-19,216,4.03,"5,364"
1693,WY,2020-09-19,34,2.22,"1,511"
1694,WV,2020-09-20,216,4.05,"5,328"


In [47]:
test = fifty_states.groupby(['state'], sort=True).agg({"collection_date":"max"}).reset_index()
test

,state,collection_date
0,AK,2020-09-20
1,AL,2020-09-20
2,AR,2020-09-20
3,AZ,2020-09-20
4,CA,2020-09-20
5,CO,2020-09-20
6,CT,2020-09-20
7,DE,2020-09-20
8,FL,2020-09-20
9,GA,2020-09-20


In [48]:
test.merge(fifty_states, on=['state', 'collection_date'])

,state,collection_date,INPT Bed Estimate,PCT INPT Bed Estimate,Total Inpatient Beds
0,AK,2020-09-20,58,3.76,"1,063"
1,AL,2020-09-20,920,6.52,"14,118"
2,AR,2020-09-20,445,5.71,"7,718"
3,AZ,2020-09-20,488,4.06,"10,898"
4,CA,2020-09-20,"3,459",5.73,"60,380"
5,CO,2020-09-20,263,2.71,"9,714"
6,CT,2020-09-20,168,2.08,"8,095"
7,DE,2020-09-20,87,4.05,"2,147"
8,FL,2020-09-20,"2,746",3.67,"74,849"
9,GA,2020-09-20,"2,101",11.70,"16,757"
